In [ ]:
import os
import cv2
import face_recognition
import numpy as np
import json
import pickle
import re

DIR = "stored_faces"
MODEL_FILE = "model.pkl"
NOTES_FILE = "notes.json"

os.makedirs(DIR, exist_ok=True)
if not os.path.exists(NOTES_FILE):
    with open(NOTES_FILE, "w") as f:
        json.dump({}, f)

with open(NOTES_FILE, "r") as f:
    notes = json.load(f)

def sanitize(input_str):
    return re.sub(r'[\\/*?:"<>|]', "_", input_str)

def collect_images():
    person_name = input("Provide a name: ")
    folder_name = sanitize(person_name)
    path = os.path.join(DIR, folder_name)
    os.makedirs(path, exist_ok=True)

    capture = cv2.VideoCapture(0)
    if not capture.isOpened():
        print("Camera error.")
        return

    print("Press 'q' to end capture.")
    img_count = 0

    while True:
        success, frame = capture.read()
        if not success:
            print("Frame error.")
            break

        rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        detections = face_recognition.face_locations(rgb_img)

        for (top, right, bottom, left) in detections:
            img_count += 1
            crop = frame[top:bottom, left:right]
            cv2.imwrite(f"{path}/{img_count}.jpg", crop)
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        cv2.imshow("Image Collection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q') or img_count >= 50:
            break

    capture.release()
    cv2.destroyAllWindows()
    print(f"Collected {img_count} images for {folder_name}.")

def build_model():
    encoded_faces = []
    labels = []

    for sub_dir in os.listdir(DIR):
        person_path = os.path.join(DIR, sub_dir)
        if not os.path.isdir(person_path):
            continue

        for img in os.listdir(person_path):
            img_path = os.path.join(person_path, img)
            try:
                loaded_img = face_recognition.load_image_file(img_path)
                encodings = face_recognition.face_encodings(loaded_img)
                if encodings:
                    encoded_faces.append(encodings[0])
                    labels.append(sub_dir)
            except:
                pass

    with open(MODEL_FILE, "wb") as file:
        pickle.dump({"encodings": encoded_faces, "labels": labels}, file)

    print("Model training complete.")

def identify_faces():
    if not os.path.exists(MODEL_FILE):
        print("Model not found. Train the model first.")
        return

    with open(MODEL_FILE, "rb") as file:
        data = pickle.load(file)

    saved_encodings = data["encodings"]
    saved_labels = data["labels"]

    capture = cv2.VideoCapture(0)
    if not capture.isOpened():
        print("Camera error.")
        return

    print("Press 'q' to quit.")
    while True:
        success, frame = capture.read()
        if not success:
            print("Frame error.")
            break

        rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        detections = face_recognition.face_locations(rgb_img)
        encodings = face_recognition.face_encodings(rgb_img, detections)

        for (top, right, bottom, left), enc in zip(detections, encodings):
            matches = face_recognition.compare_faces(saved_encodings, enc, tolerance=0.6)
            label = "Unknown"

            if True in matches:
                closest = np.argmin(face_recognition.face_distance(saved_encodings, enc))
                label = saved_labels[closest]

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

            if label != "Unknown":
                print(f"Hello, {label}!")
                if label not in notes or input(f"Update note for {label}? (y/n): ").lower() == "y":
                    note = input(f"Add a note for {label}: ")
                    notes[label] = note
                    with open(NOTES_FILE, "w") as f:
                        json.dump(notes, f)

        cv2.imshow("Face Identification", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()

while True:
    print("\nOptions:")
    print("1. Collect Images")
    print("2. Train Model")
    print("3. Identify Faces")
    print("4. Exit")

    selection = input("Choose an option: ")
    if selection == "1":
        collect_images()
    elif selection == "2":
        build_model()
    elif selection == "3":
        identify_faces()
    elif selection == "4":
        break
    else:
        print("Invalid input.")



Options:
1. Collect Images
2. Train Model
3. Identify Faces
4. Exit
Invalid input.

Options:
1. Collect Images
2. Train Model
3. Identify Faces
4. Exit
Press 'q' to end capture.
Collected 50 images for 1.

Options:
1. Collect Images
2. Train Model
3. Identify Faces
4. Exit
Model training complete.

Options:
1. Collect Images
2. Train Model
3. Identify Faces
4. Exit
Press 'q' to quit.
Hello, 1!
Hello, 1!
Hello, 1!
Hello, 1!
Hello, 1!
Hello, 1!
Hello, 1!
Hello, 1!
Hello, 1!
Hello, 1!
Hello, 1!
Hello, 1!
Hello, 1!


In [1]:
import os
import cv2
import face_recognition
import numpy as np
import json
import pickle
import re

DIR = "stored_faces"
MODEL_FILE = "model.pkl"
NOTES_FILE = "notes.json"

os.makedirs(DIR, exist_ok=True)
if not os.path.exists(NOTES_FILE):
    with open(NOTES_FILE, "w") as f:
        json.dump({}, f)

with open(NOTES_FILE, "r") as f:
    notes = json.load(f)

def sanitize(input_str):
    return re.sub(r'[\\/*?:"<>|]', "_", input_str)

def collect_images():
    person_name = input("Provide a name: ")
    folder_name = sanitize(person_name)
    path = os.path.join(DIR, folder_name)
    os.makedirs(path, exist_ok=True)

    capture = cv2.VideoCapture(0)
    if not capture.isOpened():
        print("Camera error.")
        return

    print("Press 'q' to end capture.")
    img_count = 0

    while True:
        success, frame = capture.read()
        if not success:
            print("Frame error.")
            break

        rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        detections = face_recognition.face_locations(rgb_img)

        for (top, right, bottom, left) in detections:
            img_count += 1
            crop = frame[top:bottom, left:right]
            cv2.imwrite(f"{path}/{img_count}.jpg", crop)
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        cv2.imshow("Image Collection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q') or img_count >= 50:
            break

    capture.release()
    cv2.destroyAllWindows()
    print(f"Collected {img_count} images for {folder_name}.")

def build_model():
    encoded_faces = []
    labels = []

    for sub_dir in os.listdir(DIR):
        person_path = os.path.join(DIR, sub_dir)
        if not os.path.isdir(person_path):
            continue

        for img in os.listdir(person_path):
            img_path = os.path.join(person_path, img)
            try:
                loaded_img = face_recognition.load_image_file(img_path)
                encodings = face_recognition.face_encodings(loaded_img)
                if encodings:
                    encoded_faces.append(encodings[0])
                    labels.append(sub_dir)
            except:
                pass

    with open(MODEL_FILE, "wb") as file:
        pickle.dump({"encodings": encoded_faces, "labels": labels}, file)

    print("Model training complete.")

def identify_faces():
    if not os.path.exists(MODEL_FILE):
        print("Model not found. Train the model first.")
        return

    with open(MODEL_FILE, "rb") as file:
        data = pickle.load(file)

    saved_encodings = data["encodings"]
    saved_labels = data["labels"]

    capture = cv2.VideoCapture(0)
    if not capture.isOpened():
        print("Camera error.")
        return

    print("Press 'q' to quit.")
    while True:
        success, frame = capture.read()
        if not success:
            print("Frame error.")
            break

        rgb_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        detections = face_recognition.face_locations(rgb_img)
        encodings = face_recognition.face_encodings(rgb_img, detections)

        for (top, right, bottom, left), enc in zip(detections, encodings):
            matches = face_recognition.compare_faces(saved_encodings, enc, tolerance=0.6)
            label = "Unknown"

            if True in matches:
                closest = np.argmin(face_recognition.face_distance(saved_encodings, enc))
                label = saved_labels[closest]

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

            if label != "Unknown" and label not in notes:
                print(f"Hello, {label}!")
                note = input(f"Add a note for {label}: ")
                notes[label] = note
                with open(NOTES_FILE, "w") as f:
                    json.dump(notes, f)

        cv2.imshow("Face Identification", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()

while True:
    print("\nOptions:")
    print("1. Collect Images")
    print("2. Train Model")
    print("3. Identify Faces")
    print("4. Exit")

    selection = input("Choose an option: ")
    if selection == "1":
        collect_images()
    elif selection == "2":
        build_model()
    elif selection == "3":
        identify_faces()
    elif selection == "4":
        break
    else:
        print("Invalid input.")



Options:
1. Collect Images
2. Train Model
3. Identify Faces
4. Exit
Invalid input.

Options:
1. Collect Images
2. Train Model
3. Identify Faces
4. Exit
Press 'q' to end capture.
Collected 50 images for 1.

Options:
1. Collect Images
2. Train Model
3. Identify Faces
4. Exit
Model training complete.

Options:
1. Collect Images
2. Train Model
3. Identify Faces
4. Exit
Press 'q' to quit.

Options:
1. Collect Images
2. Train Model
3. Identify Faces
4. Exit
